In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.ensemble import RandomForestRegressor

In [ ]:
latlon_data = pd.read_csv("site_locations.txt", 
                          on_bad_lines='skip', sep = '\t')

#FULLSET_HH is a folder with the HH folders of all the tower data files you want to use (HH==hourly)
path_flux = 'FULLSET_HH\\'

# read all the files with extension .csv
filenames = glob.glob(path_flux + "\*.csv")
#print('File names:', filenames)

data_dict = {}
# for loop to iterate all csv files
#commented prints are for debugging
for file in filenames:
    myfilename = file.split('\\')[-1].split('.')[0]
    myfilename = re.search("_(.*?)_", myfilename).group()
    myfilename = myfilename.split('_')[1]
    print("Reading file = ",myfilename)
 
    temp_df = pd.read_csv(file)

    #print(temp_df.head())
    #print(latlon_data[latlon_data['Site Id'] == myfilename]['Latitude (degrees)'])
    
    temp_lat = latlon_data[latlon_data['Site Id'] == myfilename]['Latitude (degrees)'].values[0]
    temp_lon = latlon_data[latlon_data['Site Id'] == myfilename]['Longitude (degrees)'].values[0]
    
    temp_df['latitude'] = temp_lat
    temp_df['longitude'] = temp_lon
    temp_df['site_name'] = myfilename
    
    #print(temp_df.head())
    
    data_dict[myfilename] = temp_df
    

In [ ]:
sub_dict0 = {}
for i in data_dict:
    print(i)
    sub_dict0[i] = data_dict[i][['TIMESTAMP_START', 'NEE_VUT_REF',  'TA_ERA', 'SW_IN_ERA','LW_IN_ERA', 'PA_ERA', 'P_ERA', 'LE_F_MDS', 'H_F_MDS', 'RH']]
    sub_dict0[i]['datetime'] = pd.to_datetime(sub_dict0[i]['TIMESTAMP_START'], format = '%Y%m%d%H%M')
    
sub_combo_df = pd.concat(sub_dict0, axis = 0)
sub_combo_df.to_csv('ameriflux_sites_combined_subset.csv')

In [ ]:
#reading in data
ameri_df = pd.read_csv('ameriflux_sites_combined_subset.csv', index_col = 1, na_values = -9999.0)
#naming 'site' column
ameri_df.rename(columns = {'Unnamed: 0':'site'}, inplace = True) 
#removing useless timestamp column
timmstmp = ameri_df.pop('TIMESTAMP_START')
# mean imputation
ameri_df['RH'] = ameri_df['RH'].fillna(ameri_df['RH'].mean())
#removing non-NAtemp sites
ameri_df = ameri_df[~ameri_df['site'].isin(['US-NGB', 'PE-QFR', 'CA-Lp1', 'AR-TF1'])]
#making datetime column datetime typer
ameri_df['datetime'] = pd.to_datetime(ameri_df['datetime'], format = '%Y-%m-%d %H:%M:%S')
#resetting index
ameri_df.reset_index(inplace = True, drop = True)
#removing half-hourly entries
ameri_df = ameri_df[ameri_df.datetime.dt.minute == 0]
#resetting index
ameri_df.reset_index(inplace = True, drop = True)

In [ ]:
reannie_df = ameri_df.copy()
reannie_df['yrmon'] = reannie_df['datetime'].dt.strftime('%Y-%m')
reannie_df['hour'] = reannie_df['datetime'].dt.strftime('%H')
reannie_df

rean_L = []

print('Beginning loop')
for site_i in reannie_df['site'].unique(): #iterating through sites
    site_sub = reannie_df[reannie_df['site'] == site_i] # getting a subset of just this site
    site_sub = site_sub.drop(['site'], axis = 1) #
    
    print(site_i)
    for yrmon in site_sub['yrmon'].unique():#getting averages for each hour over the month
        temp_df = site_sub[site_sub['yrmon'] == yrmon]
        reanned = temp_df.groupby(['hour']).mean()
        reanned['year-month'] = yrmon
        reanned['hour'] = reanned.index
        rean_L.append(reanned)


mon_hr_avs = pd.concat(rean_L)

In [ ]:
# getting column with NEE_VUT meaned 0
sub1 = mon_hr_avs[mon_hr_avs['site'] == 'CA-Cbo']
sub1.index = pd.to_datetime(sub1['year-month'], format = '%Y-%m')

month_av = pd.DataFrame(sub1['NEE_VUT_REF'].resample('1M').mean())
month_av['year-month'] = month_av.index
month_av['year-month'] = month_av['year-month'].dt.date.apply(lambda x: x.strftime('%Y-%m'))
month_av.reset_index(inplace = True, drop = True)

sub2 = sub1.reset_index(drop = True)
sub2 = sub2.merge(month_av, left_on = 'year-month', right_on = 'year-month')
sub2['NEE_VUT_REF_subt'] = sub2['NEE_VUT_REF_x'] - sub2['NEE_VUT_REF_y']

mon_hrs_av_0 = sub2
mon_hrs_av_0.to_csv('ameri_monthly_reanalysis_by_hour_mean0.csv')